In [1]:
import pandas as pd
import numpy as np
import folium
import matplotlib.pyplot as plt
import plotly.express as px
import branca.colormap as cm
from IPython.display import HTML
from folium.plugins import MousePosition
from folium import MacroElement
from jinja2 import Template

In [2]:
data_collar = pd.read_csv("Collar.csv")
data_litologi = pd.read_csv("Litologi.csv")
data_assay = pd.read_csv("Assay.csv")

In [9]:
data_collar

,Hole_id,Y,X,Z,DEPTH
0,BH01,121.513665,-2.507508,842.862,20.0
1,BH02,121.512784,-2.507511,844.527,29.0
2,BH03,121.511902,-2.507511,842.477,12.5
3,BH04,121.510981,-2.507515,836.014,27.5
4,BH05,121.510041,-2.507488,832.504,16.5
...,...,...,...,...,...
120,BH121,121.507853,-2.510636,836.000,27.0
121,BH122,121.508255,-2.510652,835.422,19.0
122,BH123,121.508693,-2.510583,838.846,15.5
123,BH124,121.509187,-2.510650,839.950,19.0


In [10]:
data_litologi

,hole_id,samp_id,dept_from,Dept_to,Litho
0,BH01,NaN,0.00,1.00,LIM
1,BH01,NaN,1.00,2.00,LIM
2,BH01,NaN,2.00,3.00,LIM
3,BH01,NaN,3.00,4.00,LIM
4,BH01,NaN,4.00,5.00,LIM
...,...,...,...,...,...
3212,BH125,NaN,10.00,10.85,SAP
3213,BH125,NaN,10.85,11.00,BR
3214,BH125,NaN,11.00,12.00,BR
3215,BH125,NaN,12.00,13.00,BR


In [18]:
print(data_assay.columns)

Index(['Hole_id', 'Samp_id', 'dept_from', 'dept_to', ' Ni', 'Co', 'Fe', 'SiO2',
       ' MgO'],
      dtype='object')


In [30]:
new_data = pd.concat([
    data_litologi[['hole_id','dept_from', 'Dept_to', 'Litho']],
    data_assay[[' Ni']]
], axis= 1)
new_data.to_csv('new_data.csv', index=False)

In [29]:
new_data

,hole_id,dept_from,Dept_to,Litho,Ni
0,BH01,0.00,1.00,LIM,0.50
1,BH01,1.00,2.00,LIM,0.72
2,BH01,2.00,3.00,LIM,0.89
3,BH01,3.00,4.00,LIM,0.98
4,BH01,4.00,5.00,LIM,0.90
...,...,...,...,...,...
3212,BH125,10.00,10.85,SAP,0.69
3213,BH125,10.85,11.00,BR,0.95
3214,BH125,11.00,12.00,BR,0.40
3215,BH125,12.00,13.00,BR,0.39


In [ ]:
# Filter only SAP rows
sap_data = new_data[new_data['Litho'] == 'SAP'].copy()

# Group by Hole_id and compute values
result = sap_data.groupby('Hole_id').agg(
    Depth_from=('dept_from', 'first'),
    Depth_to=('Dept_to', 'last')
).reset_index()

# Calculate Total_SAP as Depth_to - Depth_from
result['Total_SAP'] = result['Depth_to'] - result['Depth_from']

print(result)


    hole_id  Depth_from  Depth_to  Total_SAP
0      BH01        11.0      16.0          6
1      BH02         9.0      24.0         20
2      BH03         4.4       9.0          5
3      BH04         8.0      13.0          5
4      BH05         5.0      10.0          5
..      ...         ...       ...        ...
120    BH95         6.0      20.0         16
121    BH96        12.0      16.4          6
122    BH97         7.0      10.7          5
123    BH98        11.0      15.0          4
124    BH99         8.0      14.0          7

[125 rows x 4 columns]


In [4]:
# plt.figure(figsize=(15, 8))
# plt.scatter(data_collar['Y'], data_collar['X'], c='blue', alpha=0.5)
# plt.xlabel('Longitude')
# plt.ylabel('Latitude')
# plt.title('Peta titik dari data collar')
# plt.grid(True)
# plt.show()

In [12]:
lat = data_collar.iloc[0]['X']
lon = data_collar.iloc[0]['Y']
min_lat = data_collar['X'].min()
max_lat = data_collar['X'].max()
min_lon = data_collar['Y'].min()
max_lon = data_collar['Y'].max()

# Ambil nilai Z
z_min = data_collar['Z'].min()
z_max = data_collar['Z'].max()

# Buat peta awal
map = folium.Map(
    location=[lat, lon],
    zoom_start=17,
    control_scale= True,
    font_size='1.5rem'
    )

map.fit_bounds([[min_lat, min_lon], [max_lat, max_lon]])

# Buat colormap: kuning → merah
colormap = cm.linear.YlOrRd_09.scale(z_min, z_max)
colormap.caption = 'Z Value Scale'
# Tambahkan legend colormap ke peta
colormap.add_to(map)

mouse_position = MousePosition(
    position='bottomright',
    separator=' : ',
    prefix='Lat | Lon:',
    lat_formatter="function(num) {return L.Util.formatNum(num, 10);}",
    lng_formatter="function(num) {return L.Util.formatNum(num, 10);}",
)
mouse_position.add_to(map)

# Plot semua titik dengan warna berdasarkan Z
for _, row in data_collar.iterrows():
    z_val = row['Z']
    color = colormap(z_val)
    
    # Radius: skala ukuran sesuai nilai Z (bisa kamu sesuaikan)
    radius = 3 + ((z_val - z_min) / (z_max - z_min)) * 10  # min 3, max 13

    folium.CircleMarker(
        location=[row['X'], row['Y']],  
        radius=radius,
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=1,
        popup= (
            f"Z: {z_val:.2f}<br>"
            f"Lat: {row['X']:.5f}<br>"
            f"Lon: {row['Y']:.5f}"
        )
    ).add_to(map)
display(HTML('<div style="width: 1000px; height: 700px;">'+map._repr_html_()+'</div>'))

In [6]:
# map = folium.Map(zoom_starts = 15)

# locations = []
# for _, row in data_collar.iterrows():
#     location = [row['X'], row['Y']]
#     locations.append(location)
#     folium.CircleMarker(location=location, radius=3, color='blue', fill=True, fill_opacity=0.7).add_to(map)

# # Fit bounds ke semua lokasi
# map.fit_bounds(locations)
# map.save("collar_map.html")

In [7]:
# fig = px.scatter_mapbox(
#     data_collar,
#     lat="X",
#     lon="Y",
#     zoom=16,
#     height=600
# )

# fig.update_layout(mapbox_style="open-street-map")

# fig.show()